# Parcial Señales y Sistemas — Streamlit en Colab
Notebook único: instala dependencias, genera archivos de la app multipágina y lanza Streamlit con túnel cloudflared. Ejecuta cada sección en orden.


In [ ]:
!pip install streamlit yt-dlp pydub librosa ffmpeg-python soundfile matplotlib scipy control -q
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared
!mv cloudflared /usr/local/bin/cloudflared
!apt-get install ffmpeg -y
!mkdir -p pages logs


## Generar archivos de la app multipágina
Se crean `0_Inicio.py` y los tres archivos en `pages/`.


In [ ]:

%%writefile 0_Inicio.py
import streamlit as st

st.set_page_config(page_title="Parcial Señales y Sistemas", page_icon="📘", layout="wide")

st.title("Dashboard Parcial — Señales y Sistemas")
st.markdown(
    """
    Bienvenido al tablero interactivo del parcial de Señales y Sistemas.
    Aquí encontrarás dos ejercicios organizados en páginas independientes:

    - **Presentación:** guía rápida del proyecto y librerías usadas.
    - **Ejercicio 1 — Demodulador AM:** descarga audio de YouTube, aplica modulación/demodulación AM (DSB-SC) y permite escuchar y visualizar las señales.
    - **Ejercicio 2 — Sistema Masa-Resorte-Amortiguador:** modelado teórico, parámetros clave y simulaciones (polos/ceros, Bode, impulso, escalón y rampa).
    """
)

st.info(
    "Navega por las páginas desde el menú lateral de Streamlit. "
    "Ejecuta la app con `streamlit run 0_Inicio.py`."
)

st.markdown(
    """
    **Estructura del proyecto**

    ```
    .
    ├── 0_Inicio.py
    └── pages/
        ├── 1_Presentacion.py
        ├── 2_Ejercicio1_DemoduladorAM.py
        └── 3_Ejercicio2_MasaResorte.py
    ```

    **Dependencias principales**

    - streamlit · yt-dlp · pydub · librosa · scipy · matplotlib · soundfile
    """
)



In [ ]:

%%writefile pages/1_Presentacion.py
import streamlit as st

st.set_page_config(page_title="Presentación | Parcial Señales", layout="wide")

st.title("Presentación del Parcial")
st.markdown(
    """
    ### ¿Qué encontrarás en este dashboard?
    Este proyecto recopila los dos ejercicios del parcial de Señales y Sistemas y los organiza en un dashboard multipágina.

    - **Ejercicio 1 — Demodulador AM (DSB-SC):**
      Descarga audio desde YouTube, toma 5 segundos, lo modula en AM, lo demodula y muestra las señales en tiempo y frecuencia.
    - **Ejercicio 2 — Sistema Masa-Resorte-Amortiguador:**
      Modela el sistema mecánico y su análogo eléctrico, calcula parámetros de desempeño y simula respuestas básicas.

    ### Estructura del dashboard
    - `0_Inicio.py`: portada y atajos de navegación.
    - `pages/1_Presentacion.py`: esta explicación general.
    - `pages/2_Ejercicio1_DemoduladorAM.py`: flujo completo del demodulador AM.
    - `pages/3_Ejercicio2_MasaResorte.py`: simulador interactivo masa-resorte-amortiguador.

    ### Librerías utilizadas
    - **Streamlit:** crea la interfaz web y organiza las páginas.
    - **yt-dlp:** descarga audio desde YouTube con buena calidad.
    - **librosa:** carga audio y lo normaliza para procesarlo en NumPy.
    - **scipy.signal:** filtros sencillos, funciones de transferencia y respuestas (impulso, escalón, rampa, Bode).
    - **matplotlib:** genera las gráficas para tiempo, frecuencia y análisis del sistema.
    """
)



In [ ]:

%%writefile pages/2_Ejercicio1_DemoduladorAM.py
import numpy as np
import streamlit as st
import yt_dlp
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
from pathlib import Path
from pydub import AudioSegment
from scipy.fft import rfft, rfftfreq

st.set_page_config(page_title="Ejercicio 1 | Demodulador AM", layout="wide")

AUDIO_MENSAJE = "mensaje.wav"
AUDIO_MODULADA = "modulada.wav"
AUDIO_MEZCLA = "mezcla.wav"
AUDIO_RECUPERADA = "recuperada.wav"


def download_and_trim_audio(url: str, start_ms: int = 20_000, duration_ms: int = 5_000):
    """Descarga audio de YouTube, extrae un fragmento y lo normaliza."""
    output_stem = "cancion_descargada"
    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": output_stem + ".%(ext)s",
        "postprocessors": [{"key": "FFmpegExtractAudio", "preferredcodec": "mp3"}],
        "quiet": True,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    audio_path = Path(output_stem + ".mp3")
    audio = AudioSegment.from_file(audio_path)
    fragment = audio[start_ms : start_ms + duration_ms]
    fragment.export(AUDIO_MENSAJE, format="wav")

    samples, sr = librosa.load(AUDIO_MENSAJE, sr=None)
    samples = samples.astype(np.float32)
    samples = samples / (np.max(np.abs(samples)) + 1e-9)
    t = np.arange(len(samples)) / sr

    if audio_path.exists():
        audio_path.unlink()

    return samples, sr, t


def ideal_lowpass(signal: np.ndarray, fs: int, cutoff_hz: float) -> np.ndarray:
    """Filtro pasabajas ideal implementado en frecuencia."""
    n = len(signal)
    freqs = np.fft.rfftfreq(n, d=1 / fs)
    spectrum = np.fft.rfft(signal)
    mask = freqs <= cutoff_hz
    filtered = np.fft.irfft(spectrum * mask, n=n)
    return filtered


def modulate_am(message: np.ndarray, sr: int, carrier_freq: float, carrier_amp: float):
    t = np.arange(len(message)) / sr
    carrier = carrier_amp * np.cos(2 * np.pi * carrier_freq * t)
    modulated = message * carrier
    return modulated, carrier


def demodulate_am(modulated: np.ndarray, sr: int, carrier_freq: float, carrier_amp: float, cutoff_hz: float):
    t = np.arange(len(modulated)) / sr
    local_carrier = np.cos(2 * np.pi * carrier_freq * t)
    mixed = modulated * local_carrier
    recovered = (2 / carrier_amp) * ideal_lowpass(mixed, sr, cutoff_hz)
    return mixed, recovered


def save_audio(path: str, data: np.ndarray, sr: int) -> None:
    sf.write(path, data, sr)


def plot_time_signals(t: np.ndarray, signals):
    fig, axes = plt.subplots(len(signals), 1, figsize=(11, 7), sharex=True)
    for ax, (label, sig) in zip(axes, signals):
        ax.plot(t, sig)
        ax.set_title(label)
        ax.set_ylabel("Amplitud")
        ax.grid(True, alpha=0.3)
    axes[-1].set_xlabel("Tiempo [s]")
    fig.tight_layout()
    return fig


def plot_frequency(signals, sr: int):
    fig, axes = plt.subplots(len(signals), 1, figsize=(11, 7), sharex=True)
    for ax, (label, sig) in zip(axes, signals):
        n = len(sig)
        freqs = rfftfreq(n, 1 / sr)
        spectrum = np.abs(rfft(sig)) / n
        ax.plot(freqs, spectrum)
        ax.set_title(f"Espectro de {label}")
        ax.set_ylabel("Magnitud")
        ax.grid(True, alpha=0.3)
    axes[-1].set_xlabel("Frecuencia [Hz]")
    fig.tight_layout()
    return fig


st.title("Ejercicio 1 — Demodulador AM (DSB-SC)")
st.markdown(
    """
    Este ejercicio consiste en descargar audio desde YouTube, extraer 5 segundos,
    modularlo en AM (DSB-SC), demodularlo y analizarlo en tiempo y frecuencia.
    """
)

default_url = "https://www.youtube.com/watch?v=0ElD2qJ5ZoU"
url = st.text_input("URL de YouTube", value=default_url)

if st.button("Procesar audio"):
    with st.spinner("Descargando, modulando y demodulando..."):
        mensaje, sr, t = download_and_trim_audio(url)
        modulada, _ = modulate_am(mensaje, sr, carrier_freq=10_000, carrier_amp=1.0)
        mezcla, recuperada = demodulate_am(modulada, sr, carrier_freq=10_000, carrier_amp=1.0, cutoff_hz=4_000)

        save_audio(AUDIO_MENSAJE, mensaje, sr)
        save_audio(AUDIO_MODULADA, modulada, sr)
        save_audio(AUDIO_MEZCLA, mezcla, sr)
        save_audio(AUDIO_RECUPERADA, recuperada, sr)

        tab_audios, tab_tiempo, tab_freq = st.tabs(["Audios", "Tiempo", "Frecuencia"])

        with tab_audios:
            st.subheader("Señales procesadas")
            st.audio(AUDIO_MENSAJE, format="audio/wav")
            st.audio(AUDIO_MODULADA, format="audio/wav")
            st.audio(AUDIO_MEZCLA, format="audio/wav")
            st.audio(AUDIO_RECUPERADA, format="audio/wav")

        with tab_tiempo:
            fig_time = plot_time_signals(
                t,
                [
                    ("Mensaje original", mensaje),
                    ("Señal modulada", modulada),
                    ("Mezcla (multiplicador)", mezcla),
                    ("Señal recuperada", recuperada),
                ],
            )
            st.pyplot(fig_time, clear_figure=True)

        with tab_freq:
            fig_freq = plot_frequency(
                [
                    ("Mensaje original", mensaje),
                    ("Señal modulada", modulada),
                    ("Mezcla", mezcla),
                    ("Señal recuperada", recuperada),
                ],
                sr=sr,
            )
            st.pyplot(fig_freq, clear_figure=True)



In [ ]:
%%writefile pages/3_Ejercicio2_MasaResorte.py
import numpy as np
import matplotlib.pyplot as plt
import streamlit as st
from scipy import signal
import control as ctrl


st.set_page_config(page_title="Ejercicio 2 | Masa-Resorte", layout="wide")


def damping_parameters(m: float, c: float, k: float):
    """Calcula zeta, frecuencias y tiempos característicos."""
    wn = np.sqrt(k / m)
    zeta = c / (2 * np.sqrt(k * m))
    wd = wn * np.sqrt(max(0.0, 1 - zeta**2))
    tp = np.pi / wd if wd > 0 else np.nan
    tr = 1.8 / wn if wn > 0 else np.nan
    ts = 4 / (zeta * wn) if zeta > 0 else np.nan
    return zeta, wn, wd, tp, tr, ts


def build_time_vector(wn: float, zeta: float):
    """Malla temporal adaptable a sistemas poco amortiguados."""
    base = max(5.0, 8.0 / max(wn, 0.2))
    if zeta < 0.3:
        base *= 2
    return np.linspace(0, base, 900)


def build_system(m: float, c: float, k: float):
    """Genera el sistema LTI de planta G(s)=1/(m s^2 + c s + k)."""
    num = [1.0]
    den = [m, c, k]
    return signal.lti(num, den)


def plot_pole_zero(num, den):
    z, p, _ = signal.tf2zpk(num, den)
    fig, ax = plt.subplots()
    if len(p) > 0:
        ax.scatter(np.real(p), np.imag(p), marker="x", color="tab:red", label="Polos")
    if len(z) > 0:
        ax.scatter(np.real(z), np.imag(z), marker="o", color="tab:blue", label="Ceros")
    ax.axhline(0, color="gray", linewidth=0.8)
    ax.axvline(0, color="gray", linewidth=0.8)
    ax.set_xlabel("Parte real")
    ax.set_ylabel("Parte imaginaria")
    ax.set_title("Diagrama de polos y ceros")
    ax.grid(True, alpha=0.4)
    ax.legend()
    fig.tight_layout()
    return fig


def plot_bode(sys):
    w, mag, phase = signal.bode(sys)
    fig, axes = plt.subplots(2, 1, figsize=(7, 6), sharex=True)
    axes[0].semilogx(w, mag)
    axes[0].set_ylabel("Magnitud [dB]")
    axes[0].set_title("Bode — Magnitud")
    axes[0].grid(True, which="both", ls=":")
    axes[1].semilogx(w, phase, color="tab:orange")
    axes[1].set_ylabel("Fase [deg]")
    axes[1].set_xlabel("Frecuencia [rad/s]")
    axes[1].set_title("Bode — Fase")
    axes[1].grid(True, which="both", ls=":")
    fig.tight_layout()
    return fig


def plot_response(title: str, t: np.ndarray, y: np.ndarray):
    fig, ax = plt.subplots()
    ax.plot(t, y)
    ax.set_title(title)
    ax.set_xlabel("Tiempo [s]")
    ax.set_ylabel("Salida")
    ax.grid(True, alpha=0.3)
    fig.tight_layout()
    return fig


st.title("Ejercicio 2 — Sistema Masa-Resorte-Amortiguador")

st.markdown(
    r"""
### Modelación teórica
- Ecuación diferencial:  \( m y''(t) + c y'(t) + k y(t) = f_e(t) \)
- Transformada de Laplace: \( m s^2 Y(s) + c s Y(s) + k Y(s) = F_e(s) \)
- Función de transferencia mecánica: \( H(s) = \dfrac{Y(s)}{F_e(s)} = \dfrac{1}{m s^2 + c s + k} \)
- Equivalente eléctrico: \( H(s) = \dfrac{V_o(s)}{V_i(s)} = \dfrac{1}{L C s^2 + (L/R) s + 1} \)
- Analogías: \( m \leftrightarrow L \), \( c \leftrightarrow R \), \( k \leftrightarrow 1/C \)
"""
)

zeta_options = {
    "Subamortiguado (ζ=0.5)": 0.5,
    "Crítico (ζ=1)": 1.0,
    "Sobreamortiguado (ζ=2)": 2.0,
}

with st.sidebar:
    st.header("Parámetros del sistema")
    m = st.number_input("m (masa)", min_value=0.1, value=1.0, step=0.1)
    k = st.number_input("k (constante del resorte)", min_value=0.1, value=20.0, step=0.5)
    zeta_label = st.radio("Tipo de sistema", options=list(zeta_options.keys()), index=0)
    zeta_target = zeta_options[zeta_label]
    c_auto = 2 * zeta_target * np.sqrt(m * k)
    c_manual = st.number_input("c (coeficiente de amortiguación) — manual", min_value=0.0, value=1.0, step=0.1)
    use_manual_c = st.checkbox("Usar c manual (ignorar selección de ζ)", value=False)
    c_used = c_manual if use_manual_c else c_auto
    st.caption(f"c sugerido por ζ={zeta_target:.2f}: {c_auto:.3f}  |  c usado: {c_used:.3f}")
    simular = st.button("Simular sistema")


if simular:
    zeta, wn, wd, tp, tr, ts = damping_parameters(m, c_used, k)
    num = [1.0]
    den = [m, c_used, k]
    sistema = build_system(m, c_used, k)
    t = build_time_vector(wn, zeta)

    # Respuestas en lazo abierto
    t_imp, y_imp = signal.impulse(sistema, T=t)
    t_step, y_step = signal.step(sistema, T=t)
    ramp_input = t
    t_ramp, y_ramp, _ = sistema.output(U=ramp_input, T=t)

    # Respuesta en lazo cerrado con realimentación unitaria
    g = ctrl.tf([1], [m, c_used, k])
    cl = ctrl.feedback(g, 1)
    try:
        t_cl, y_cl = ctrl.step_response(cl, T=t)
    except AttributeError:
        t_cl, y_cl = ctrl.step(cl, T=t)

    tab_graficas, tab_datos, tab_explicacion = st.tabs(["Gráficas", "Datos", "Explicación"])

    with tab_graficas:
        col1, col2 = st.columns(2)
        with col1:
            st.pyplot(plot_pole_zero(num, den), clear_figure=True)
            st.pyplot(plot_response("Respuesta al impulso (lazo abierto)", t_imp, y_imp), clear_figure=True)
        with col2:
            st.pyplot(plot_bode(sistema), clear_figure=True)
            st.pyplot(plot_response("Respuesta al escalón (lazo abierto)", t_step, y_step), clear_figure=True)
        st.pyplot(plot_response("Respuesta a la rampa (lazo abierto)", t_ramp, y_ramp), clear_figure=True)
        st.pyplot(
            plot_response(
                "Escalón en lazo cerrado (retroalimentación unitaria)",
                np.array(t_cl),
                np.array(y_cl),
            ),
            clear_figure=True,
        )

    with tab_datos:
        st.subheader("Parámetros calculados")
        st.write(
            {
                "m": m,
                "k": k,
                "c manual": c_manual,
                "c sugerido": round(c_auto, 4),
                "c usado": round(c_used, 4),
                "ζ (amortiguamiento)": round(zeta, 4),
                "ω_n [rad/s]": round(wn, 4),
                "ω_d [rad/s]": round(wd, 4),
                "T_p [s]": round(tp, 4),
                "T_r [s]": round(tr, 4),
                "T_s [s]": round(ts, 4),
            }
        )
        st.caption(
            f"Clasificación automática: {'Subamortiguado' if zeta < 1 else 'Crítico' if np.isclose(zeta, 1, atol=1e-2) else 'Sobreamortiguado'}."
        )

    with tab_explicacion:
        st.markdown(
            """
            - **Selector de tipo de sistema** ajusta ζ y calcula automáticamente c = 2 ζ √(m k). Puedes forzar un valor manual con la casilla correspondiente.
            - **Lazo abierto:** se muestran polos/ceros, Bode, impulso, escalón y rampa para G(s).
            - **Lazo cerrado:** se agrega la respuesta al escalón con realimentación unitaria para comparar desempeño.
            - Cambia m, k o ζ para ver cómo se modifican las respuestas y los tiempos característicos.
            """
        )



## Lanzar Streamlit con cloudflared
Detiene procesos previos, arranca Streamlit y muestra la URL pública.


In [ ]:
print("Deteniendo y limpiando procesos anteriores...")
!pkill -f streamlit || true
!pkill -f cloudflared || true


In [ ]:
print("Lanzando Streamlit y túnel...")
!streamlit run 0_Inicio.py --server.port 8501 --server.address 0.0.0.0 &>/content/logs.txt &
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &
import time, re, pathlib

time.sleep(12)
log_path = pathlib.Path('/content/cloudflared.log')
if log_path.exists():
    data = log_path.read_text()
    m = re.search(r"https://[-\w\.]+\.trycloudflare\.com", data)
    if m:
        print('✅ Tu app está disponible en:', m.group(0))
    else:
        print('❌ No se encontró el enlace. Revisa /content/cloudflared.log')
else:
    print('❌ cloudflared no generó log')

